## Build a RAG System with Langchain and ChromaDB

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Langchain Imports

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# vector stores
from langchain_community.vectorstores import Chroma

# Utility imports
import numpy as np
from typing import List

d:\RAG\RAG-Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample_docs = [
    """
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, advancements in deep learning and neural networks are driving significant progress in areas such as computer vision, natural language processing, and autonomous systems.
    """,

    """
    Natural Language Processing (NLP)

    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as text classification, sentiment analysis, named entity recognition, machine translation, and speech recognition. Techniques used in NLP include tokenization, part-of-speech tagging, syntactic parsing, and semantic analysis. Recent advancements in NLP have been driven by deep learning models such as transformers, which have significantly improved the performance of language models like BERT and GPT. Applications of NLP are widespread, including chatbots, virtual assistants, automated customer support, and content recommendation systems. As NLP continues to advance, it plays a crucial role in enhancing human-computer interaction and enabling more natural communication with technology.
    """,
    """
    Deep Learning and Neural Networks

    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that can automatically learn hierarchical representations of data. Key architectures in deep learning include convolutional neural networks (CNNs) for image processing, recurrent neural networks (RNNs) for sequential data, and transformers for language modeling. Training deep learning models typically requires large datasets and significant computational resources, often utilizing GPUs for efficient processing. The success of deep learning has led to breakthroughs in applications such as autonomous vehicles, medical image analysis, and real-time language translation. As research continues, deep learning is expected to drive further innovations in artificial intelligence and its applications across various domains.
    """
]

sample_docs

['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, advancements in deep learning and neural networks 

In [4]:
### save sample documents to files
import tempfile
# temp_dir= tempfile.mkdtemp()

for i, doc in enumerate(sample_docs):
    # with open(os.path.join(temp_dir, f"doc_{i}.txt"), "w") as f:
    #     f.write(doc)
    with open(f"doc_{i}.txt", "w") as f:
        f.write(doc.strip())

print(f"Sample documents saved to root")

Sample documents saved to root


## Document Loading

In [5]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('.', glob='doc_*.txt', loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})

documents = loader.load()

print(f"Loaded {len(documents)} documents.")
print("First document content:")
print(documents[0].page_content[:200])  # print first 200 characters of the first document

Loaded 3 documents.
First document content:
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al


In [6]:
documents

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision trees, support vector machines, neural networks, and clustering techniques. As the field continues to evolve, 

## Document Splitting

In [7]:
# Initialize Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "] # Hierarchy of separators
)

chunks = text_splitter.split_documents(documents)

chunks

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

## Initialize ChromaDB Vector store and store the chunks

In [8]:
## Create a ChromaDB Vector Store

persist_directory = './chroma_db'

## Initialize ChromaDB with HuggingFace Embeddings
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    persist_directory=persist_directory,
    collection_name="rag_collection"
)

print(f"Vector store created with {vectorstore._collection.count()} vectors.")
print(f"Vectors persisted in directory: {persist_directory}")

Vector store created with 9 vectors.
Vectors persisted in directory: ./chroma_db


### Test Similarity search

In [9]:
query = "What are types of machine learning?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

In [10]:
query = "What is NLP?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_1.txt'}, page_content='Natural Language Processing (NLP)\n\n    Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves the development of algorithms and models that enable machines to understand, interpret, and generate human language in a way that is meaningful. NLP encompasses various tasks such as text classification, sentiment analysis, named entity recognition, machine translation, and speech recognition. Techniques'),
 Document(metadata={'source': 'doc_1.txt'}, page_content='systems. As NLP continues to advance, it plays a crucial role in enhancing human-computer interaction and enabling more natural communication with technology.'),
 Document(metadata={'source': 'doc_1.txt'}, page_content='translation, and speech recognition. Techniques used in NLP include tokenization, part-of-speech tagging, syntactic parsing, and semantic analysis. Recent adva

In [11]:
query = "What is deep learning?"

similar_docs = vectorstore.similarity_search(query, k=3)

similar_docs

[Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that'),
 Document(metadata={'source': 'doc_2.txt'}, page_content='by enabling the development of models that can automatically learn hierarchical representations of data. Key architectures in deep learning include convolutional neural networks (CNNs) for image processing, recurrent neural networks (RNNs) for sequential data, and transformers for language modeling. Training deep learning models typically requires large datasets and significa

### Advanced Similarity Search with Scores

In [12]:
result_scores= vectorstore.similarity_search_with_score(query, k=3)
result_scores

[(Document(metadata={'source': 'doc_2.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural networks are inspired by the structure and function of the human brain, consisting of interconnected nodes (neurons) that process information. Deep learning has revolutionized various fields, including computer vision, natural language processing, and speech recognition, by enabling the development of models that'),
  0.5512487888336182),
 (Document(metadata={'source': 'doc_2.txt'}, page_content='by enabling the development of models that can automatically learn hierarchical representations of data. Key architectures in deep learning include convolutional neural networks (CNNs) for image processing, recurrent neural networks (RNNs) for sequential data, and transformers for language modeling. Training deep learning models typically requires lar

### Understanding Similarity Scores

The similarity score represents how closely related a document chunk is to your query, The scoring depends on the distance metric used:

ChromaDB by default uses L2 distance (Euclidean distance)
- Lower scores =  MORE similar (closer in vector space)
- Score of 0 =  identical vectors
- Typical range: 0 to 2 (but can be higher)

Cosine similarity (if configured):

- Higher scores =  MORE similar
- Range: -1 to 1 (1 being identical)

## Initialize LLM, RAG Chain, Prompt Tempate, Query the RAG system

In [13]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="mistral:latest")

In [14]:
test_response =  llm.invoke("What is Large Language Models?")

test_response

AIMessage(content=' Large Language Models (LLMs) are a type of artificial intelligence (AI) model designed to process and generate human-like text. They are trained on vast amounts of internet text, learning to predict the next word in a sentence based on context. These models can answer questions, write essays, summarize texts, translate languages, and even carry out conversations. Examples include models like me, such as Megatron, T5, BERT, and GPT-3 from companies like Hugging Face, Google, and OpenAI.\n\nLarge Language Models are a subset of Transformer models, which were introduced in 2017 by Vaswani et al. The transformer model architecture is designed to handle sequences of data and has shown great success in natural language processing tasks, especially in understanding the context and meaning behind text. These models have made significant advancements in the field of AI and natural language understanding (NLU).', additional_kwargs={}, response_metadata={'model': 'mistral:late

In [15]:
# Alternative waqy to initialize chat models

from langchain.chat_models.base import init_chat_model

chat_model = init_chat_model("ollama:mistral:latest")
#llm = init_chat_model("groq:")

chat_model

ChatOllama(model='mistral:latest')

In [16]:
chat_model.invoke("Explain the concept of reinforcement learning.")

AIMessage(content=" Reinforcement Learning (RL) is a type of machine learning where an agent learns to make decisions by interacting with its environment. The goal is for the agent to learn a policy, which is a strategy that maps states to actions, enabling it to maximize some notion of cumulative reward over time.\n\nHere's a simplified breakdown:\n\n1. **Agent**: This is the learning entity that interacts with the environment. It can be anything from a robot arm in a factory, an autonomous car, or even an AI playing a video game.\n\n2. **Environment**: This is the world in which the agent operates. The environment could be a physical space (like a room), a virtual space (like a video game), or even an abstract concept (like financial markets).\n\n3. **State**: At any given time, the environment is in some specific condition or configuration. This state is observed by the agent and used to decide its next action.\n\n4. **Action**: The agent selects an action based on its current state

## Modern RAG Chain

In [17]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents.stuff import create_stuff_documents_chain

In [18]:
## Convert vectorstore to a retriever

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000221E98BEE50>, search_kwargs={'k': 3})

In [19]:
##Create Prompt Template
system_prompt = """You are an assistant for question-answering tasks.
Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [20]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [21]:
## Create Documents Chain
documents_chain = create_stuff_documents_chain(
    llm=chat_model,
    prompt=prompt
)

documents_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOllama(model='mistral:latest')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_document

This chain:

- Takes retrieved documents
- "Stuffs" them into prompt's {context} placeholder.
- Sends the complete prompt to the LLM.
- Returns the LLM's response.

In [22]:
### Create the Final RAG Chain
rag_chain = create_retrieval_chain(retriever,documents_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000221E98BEE50>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the context provided to answer the question as accurately as possible

Using this create_retrieval_chain we are able to another retreval layer to our stuff document chain which uses the retriever to fetch relavant documents from our vectorstore and sends it to the stuff chain which sends it to the LLM.

In [23]:
response = rag_chain.invoke({"input": "Explain the different types of machine learning."})

response

{'input': 'Explain the different types of machine learning.',
 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
  Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, market

In [24]:
response["answer"]

' The three main types of machine learning are:\n1. Supervised Learning: Uses labeled data to train models for specific tasks, such as image classification or speech recognition.\n2. Unsupervised Learning: Finds patterns in unlabeled data without explicit guidance, useful for clustering and anomaly detection.\n3. Reinforcement Learning: Trains agents to make sequences of decisions by rewarding desired behaviors, commonly used for navigation and gaming.\n\nDeep learning, a subset of machine learning, utilizes neural networks with multiple layers to model complex patterns in data. This has significantly impacted various fields like computer vision, natural language processing, and speech recognition.'

In [25]:
# Function to query the modern RAG system

def query_rag_modern(question: str) -> str:
    print(f"Question: {question}")
    print("-"*50)

    # Using create_retrieval_chain approach
    result = rag_chain.invoke({"input": question})

    print(f"Answer: {result}")
    print("\nRetrieved Context:")
    for i, doc in enumerate(result['context']):
        print(f"\n ----- Source {i+1} -----")
        print(doc.page_content[:200]+"...")

    return result['answer']

test_questions = [
    "What are three types of machine learning?",
    "What is Deep Learning and how does it relate to Neural Networks?",
    "What are CNNs used for?"
]

for question in test_questions:
    query_rag_modern(question)
    print("\n"+"="*80+"\n")

Question: What are three types of machine learning?
--------------------------------------------------
Answer: {'input': 'What are three types of machine learning?', 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'), Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. A

# Creating Rag Chain Alternative - Using LCEL

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


In [27]:
#Create a custom Prompt
custom_prompt = ChatPromptTemplate.from_template(
    """Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}

Question: {question}

Answer:"""
)

custom_prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])

In [28]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000221E98BEE50>, search_kwargs={'k': 3})

In [29]:
## Format the output documents for the prompt
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

In [30]:
## Build the RAG Chain using LCEL
rag_chain_lcel = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()  # we use runnable passthrough to accept question input during runtime
    }
    | custom_prompt
    | chat_model
    | StrOutputParser()
)

rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000221E98BEE50>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the context provided to answer the question as accurately as possible.\nIf the context does not contain the answer, respond with 'I don't know'.\nUse three sentences maximum to answer the question and keep it concise.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])
| ChatOllama(model='mistral:latest')
| StrOutputParser()

In [31]:
response = rag_chain_lcel.invoke("Explain the different types of machine learning.")
response

' The three main types of machine learning are Supervised Learning, Unsupervised Learning, and Reinforcement Learning. In Supervised Learning, labeled data is used to train models, while in Unsupervised Learning, patterns are found in unlabeled data. Reinforcement Learning involves training agents to make sequences of decisions by rewarding desired behaviors. Deep Learning, a subset of machine learning, utilizes neural networks with multiple layers to model complex patterns in data.'

In [32]:
context = retriever.invoke("Explain the different types of machine learning.")
context

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

In [33]:
# Query using the LCEL RAG Chain

def query_rag_lcel(question: str) -> str:
    print(f"Question: {question}")
    print("-"*50)

    # Using create_retrieval_chain approach
    result = rag_chain_lcel.invoke(question)

    print(f"Answer: {result}")
    
    docs = retriever.invoke(question)
    print("\nSource Context:")

    for i, doc in enumerate(docs):
        print(f"\n ----- Source {i+1} -----")
        print(doc.page_content[:200]+"...")


test_questions = [
    "What are three types of machine learning?",
    "What is Deep Learning and how does it relate to Neural Networks?",
    "What are CNNs used for?"
]


In [34]:
for question in test_questions:
    query_rag_lcel(question)
    print("\n"+"="*80+"\n")

Question: What are three types of machine learning?
--------------------------------------------------
Answer:  The three types of machine learning are Supervised Learning, Unsupervised Learning, and Reinforcement Learning. In Supervised Learning, models are trained using labeled data. Unsupervised Learning finds patterns in unlabeled data. Lastly, Reinforcement Learning trains agents to make sequences of decisions by rewarding desired behaviors.

Source Context:

 ----- Source 1 -----
Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various al...

 ----- Source 2 -----
labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behavior...

 ----- Source 3 -----
techniques. As the field continues to evolve, advanceme

# Adding new documents to existing vector store

In [35]:
vectorstore

In [36]:
# Adding new documents to existing vector store
new_document = """
Convolutional Neural Networks (CNNs) in detail

Convolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They utilize convolutional layers to automatically and adaptively learn spatial hierarchies of features from input data. CNNs are widely used in computer vision tasks, including image classification, object detection, and image segmentation, due to their ability to capture local patterns and reduce the number of parameters compared to fully connected networks. 

They have proven to be highly effective in various applications, such as facial recognition, medical image analysis, and autonomous driving, making them a fundamental component of modern AI systems.
"""

In [37]:
chunks

[Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructions. The key types of machine learning include supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised'),
 Document(metadata={'source': 'doc_0.txt'}, page_content='labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors. Applications of machine learning span across various industries, including healthcare, finance, marketing, and more. Common algorithms used in machine learning include decision 

In [38]:
from langchain_core.documents import Document

new_doc = Document(page_content=new_document,
                   metadata={"source": "manual_addition", "topic": "CNNs"})

In [39]:
new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'CNNs'}, page_content='\nConvolutional Neural Networks (CNNs) in detail\n\nConvolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They utilize convolutional layers to automatically and adaptively learn spatial hierarchies of features from input data. CNNs are widely used in computer vision tasks, including image classification, object detection, and image segmentation, due to their ability to capture local patterns and reduce the number of parameters compared to fully connected networks. \n\nThey have proven to be highly effective in various applications, such as facial recognition, medical image analysis, and autonomous driving, making them a fundamental component of modern AI systems.\n')

In [40]:
new_chunks = text_splitter.split_documents([new_doc])

In [41]:
# Add new chunks to existing vector store
vectorstore.add_documents(new_chunks)

['4eb80870-0e4c-4b2a-b281-ef47120152ba',
 'eb813304-1e0b-4ec9-874a-e73920661d78']

In [42]:
print(f"Added {len(new_chunks)} new chunks to the vector store.")
print(f"Updated vector store now has {vectorstore._collection.count()} vectors.")

Added 2 new chunks to the vector store.
Updated vector store now has 11 vectors.


In [43]:
#Query after adding new documents

query = "What are Convolutional Neural Networks?"
query_rag_lcel(query)

Question: What are Convolutional Neural Networks?
--------------------------------------------------
Answer:  Convolutional Neural Networks (CNNs) are a type of deep learning model that specializes in processing grid-structured data, such as images. They automatically and adaptively learn spatial hierarchies of features from input data using convolutional layers. Widely used in computer vision tasks like image classification, object detection, and image segmentation due to their ability to capture local patterns and reduce dimensionality.

Source Context:

 ----- Source 1 -----
Convolutional Neural Networks (CNNs) in detail

Convolutional Neural Networks (CNNs) are a class of deep learning models specifically designed for processing structured grid data, such as images. They...

 ----- Source 2 -----
Deep Learning and Neural Networks

    Deep learning is a subset of machine learning that focuses on using neural networks with multiple layers to model complex patterns in data. Neural ne

# Advanced RAG Techniques- Conversational Memory

- create_history_aware_retriever: Makes retriever understand conversation context
- MessagesPlaceholder: Placeholder for chat history in prompts
- HumanMessage, AIMessage: Structured Message types for conversation history

In [44]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [45]:
# create a prompt with chat history placeholder

contextualized_q_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return the question as is.
"""

contextualized_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualized_q_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

In [46]:
## create a history aware retriever
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualized_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000221E98BEE50>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, T

In [47]:
# Create a new document chain with history


qa_system_prompt = """You are an assistant for question-answering tasks.
Use the context provided to answer the question as accurately as possible.
If the context does not contain the answer, respond with 'I don't know'.
Use three sentences maximum to answer the question and keep it concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

queston_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create conversational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    queston_answer_chain
)

print("Conversational RAG Chain created.")

Conversational RAG Chain created.


In [48]:
chat_history = []
#First Query
response1 = conversational_rag_chain.invoke({
    "input": "What is machine learning?",
    "chat_history": chat_history
})

print(f"Question: What is machine learning?")
print(f"Answer: {response1['answer']}")

Question: What is machine learning?
Answer:  Machine Learning is a subset of Artificial Intelligence that enables computers to learn from data without being explicitly programmed. It uses algorithms and statistical models to make decisions or perform specific tasks. The key types include supervised learning (with labeled data), unsupervised learning (without labeled data), and reinforcement learning (involving rewards for desired behaviors).


In [49]:
chat_history

chat_history.extend([HumanMessage(content="What is machine learning?"),
                    AIMessage(content=response1['answer'])])

In [50]:
# Follow-up Query
response2 = conversational_rag_chain.invoke({
    "input": "What are its main types?",
    "chat_history": chat_history
})

print(f"Question: What are its main types?")
print(f"Answer: {response2['answer']}")

response2

Question: What are its main types?
Answer:  The main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning. Supervised learning uses labeled data to train models, while unsupervised learning finds patterns in unlabeled data. Reinforcement learning involves training agents to make sequences of decisions by rewarding desired behaviors.


{'input': 'What are its main types?',
 'chat_history': [HumanMessage(content='What is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content=' Machine Learning is a subset of Artificial Intelligence that enables computers to learn from data without being explicitly programmed. It uses algorithms and statistical models to make decisions or perform specific tasks. The key types include supervised learning (with labeled data), unsupervised learning (without labeled data), and reinforcement learning (involving rewards for desired behaviors).', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'source': 'doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves various algorithms and statistical models that enable computers to perform specific tasks without explicit instructi